# morph-var 3

## Build, train and test a landform profile shape classifier using user-chosen classes

This is the third of five notebooks outlining the algorithm we used to quantify the morphologic variability of a linear landform. In this notebook, we first list profiles selected from those processed in morph-var 2 and manually grouped into six morphologic categories with a view towards geologic process. Then, we use principal component analysis with singular value decomposition (SVD) to quantitatively distinguish between morphologic classes. We follow this by employing the support vector machine (SVM) method to build a supervised classifier, using the principal-component coordinates of the classified profiles in principal component space as a training set. Classification performance was assessed using 5-fold cross validation (81% accuracy). The "eigenfaces" of the 

## Packages and libraries

In [ ]:
import numpy as np
import pandas as pd
from geomdl import BSpline, utilities
from scipy import linalg
import itertools
import matplotlib.pyplot as plt
import os
from sklearn import svm
import sklearn
from sklearn import preprocessing
from mpl_toolkits.mplot3d.axes3d import Axes3D

In [ ]:
%matplotlib qt

## Define landform profile morphologic classes

Using field observations and expert knowledge, define morphologic classes and select a dataset of manually classified profiles to train the classifier.

In [ ]:
#Specify the number of morphologic classes
n_classes = 6

#For each class, list the filenames (strings) of the selected profiles to represent each class.
class1='VogarA_cropnorm_146.txt','VogarA_cropnorm_95.txt','VogarA_cropnorm_98.txt','VogarA_cropnorm_2.txt','Stora_Aragja_cropnorm_1.txt','Stora_Aragja_cropnorm_2.txt','Stora_Aragja_cropnorm_4.txt','Stora_Aragja_cropnorm_32.txt','Stora_Aragja_cropnorm_40.txt','Sodulholagja_cropnorm_18.txt','Sodulholagja_cropnorm_28.txt','RHPS_cropnorm_190.txt','RHPS_cropnorm_193.txt','RHPS_cropnorm_195.txt','RHPS_cropnorm_208.txt','VogarA_cropnorm_68.txt','RHPS_cropnorm_210.txt'
class2='VogarA_cropnorm_129.txt','VogarA_cropnorm_131.txt','VogarA_cropnorm_133.txt','VogarA_cropnorm_134.txt','VogarA_cropnorm_135.txt','VogarA_cropnorm_137.txt','VogarA_cropnorm_138.txt','VogarA_cropnorm_139.txt','VogarA_cropnorm_140.txt','VogarA_cropnorm_38.txt','VogarA_cropnorm_50.txt','VogarA_cropnorm_47.txt','VogarA_cropnorm_46.txt','VogarA_cropnorm_57.txt','VogarA_cropnorm_56.txt','VogarA_cropnorm_55.txt','VogarA_cropnorm_53.txt','VogarA_cropnorm_51.txt','Stora_Aragja_cropnorm_34.txt','Stora_Aragja_cropnorm_35.txt','Stora_Aragja_cropnorm_54.txt','Stora_Aragja_cropnorm_55.txt'
class3='LVBD-A_4_cropnorm_300.txt','LVBD-A_4_cropnorm_302.txt','LVBD-A_4_cropnorm_316.txt','LVBD-A_4_cropnorm_335.txt','LVBD-A_4_cropnorm_317.txt','LVBD-A_4_cropnorm_340.txt','LVBD-A_4_cropnorm_355.txt','LVBD-A_4_cropnorm_365.txt','LVBD-A_4_cropnorm_375.txt','LVBD-A_4_cropnorm_385.txt','LVBD-A_4_cropnorm_390.txt','LVBD-A_4_cropnorm_535.txt','LVBD-A_4_cropnorm_540.txt','LVBD-A_4_cropnorm_590.txt','LVBD-A_4_cropnorm_630.txt','LVBD-A_4_cropnorm_760.txt','LVBD-A_4_cropnorm_770.txt','VogarA_cropnorm_118.txt','VogarA_cropnorm_78.txt','Stora_Aragja_cropnorm_9.txt','Stora_Aragja_cropnorm_15.txt'
class4='VogarA_cropnorm_30.txt','VogarA_cropnorm_33.txt','VogarA_cropnorm_35.txt','VogarA_cropnorm_41.txt','VogarA_cropnorm_61.txt','VogarA_cropnorm_63.txt','VogarA_cropnorm_64.txt','LVBD-A_1_cropnorm_136.txt','LVBD-A_1_cropnorm_145.txt','LVBD-A_1_cropnorm_147.txt','LVBD-A_1_cropnorm_155.txt','LVBD-A_1_cropnorm_275.txt','LVBD-A_1_cropnorm_276.txt','Stora_Aragja_cropnorm_41.txt','Stora_Aragja_cropnorm_60.txt','Sodulholagja_cropnorm_37.txt','Sodulholagja_cropnorm_38.txt','RHPS_cropnorm_140.txt','RHPS_cropnorm_430.txt'
class5='LVBD-A_1_cropnorm_96.txt','LVBD-A_1_cropnorm_410.txt','LVBD-A_1_cropnorm_570.txt','LVBD-A_1_cropnorm_575.txt','LVBD-A_1_cropnorm_650.txt','LVBD-A_1_cropnorm_690.txt','LVBD-A_4_cropnorm_0.txt','LVBD-A_4_cropnorm_5.txt','LVBD-A_4_cropnorm_7.txt','LVBD-A_4_cropnorm_11.txt','LVBD-A_4_cropnorm_210.txt','LVBD-A_4_cropnorm_225.txt','LVBD-A_4_cropnorm_515.txt','Stora_Aragja_cropnorm_6.txt','Stora_Aragja_cropnorm_67.txt','RHPS_cropnorm_20.txt','RHPS_cropnorm_60.txt','RHPS_cropnorm_65.txt'
class6='VogarA_cropnorm_26.txt','VogarA_cropnorm_28.txt','LVBD-A_1_cropnorm_100.txt','LVBD-A_1_cropnorm_101.txt','LVBD-A_1_cropnorm_102.txt','LVBD-A_1_cropnorm_104.txt','LVBD-A_1_cropnorm_106.txt','LVBD-A_1_cropnorm_114.txt','LVBD-A_1_cropnorm_117.txt','LVBD-A_1_cropnorm_130.txt','LVBD-A_1_cropnorm_164.txt','LVBD-A_1_cropnorm_166.txt','LVBD-A_1_cropnorm_167.txt','LVBD-A_1_cropnorm_450.txt','LVBD-A_1_cropnorm_800.txt','LVBD-A_1_cropnorm_854.txt','LVBD-A_4_cropnorm_150.txt','LVBD-A_4_cropnorm_155.txt','LVBD-A_4_cropnorm_151.txt','Stora_Aragja_cropnorm_44.txt','Stora_Aragja_cropnorm_51.txt','Stora_Aragja_cropnorm_58.txt','Stora_Aragja_cropnorm_59.txt','Stora_Aragja_cropnorm_64.txt','Stora_Aragja_cropnorm_68.txt','Sodulholagja_cropnorm_26.txt','RHPS_cropnorm_397.txt','RHPS_cropnorm_500.txt'

## Load normalized landform profile data

Load the 2-D coordinates of the cropped and normalized profiles (output of process-profiles) for each class list. 

In [ ]:
# Define a function to load the coordinates from the cropped and normalized landform profiles.
def load_data(class_name,path):
    #Set path to folder with 2-D cropped and normalized profiles.
    data_x_temp = []
    data_y_temp = []
    filenames =class_name #select profiles that belong to the desired class training group.
    for file in filenames:
            # Load x-coordinate data from files matching regular expression for desired class
            data_x_temp.append(pd.read_csv(path+"/"+file).iloc[:,1].values)
            # Load y-coordinate data from files matching regular expression for desired class
            data_y_temp.append(pd.read_csv(path+"/"+file).iloc[:,2].values)
            
    # Create dataframes for x and y
    dataclass_x = pd.DataFrame(data_x_temp).transpose() 
    dataclass_y = pd.DataFrame(data_y_temp).transpose()
    dataclass_x.columns = filenames
    dataclass_y.columns = filenames

    return dataclass_x, dataclass_y

In [ ]:
#Set path to that of the output files from process-profiles
path= "/Users/cassandrabrigham/Documents/RESEARCH/2020/Code/Classifier/Final/Data2/All_2D_cropped_normalized"

In [ ]:
# Load profile data for each class
(data1_x, data1_y) = load_data(class1,path)
(data2_x, data2_y) = load_data(class2,path)
(data3_x, data3_y) = load_data(class3,path)
(data4_x, data4_y) = load_data(class4,path)
(data5_x, data5_y) = load_data(class5,path)
(data6_x, data6_y) = load_data(class6,path)

In [ ]:
#Determine length of each class list, as well as total length of training dataset.
len1 = len(data1_x.columns)
len2 = len(data2_x.columns)
len3 = len(data3_x.columns)
len4 = len(data4_x.columns)
len5 = len(data5_x.columns)
len6 = len(data6_x.columns)

total_len = len1+len2+len3+len4+len5+len6

In [ ]:
#Define 2 matrices with the positional data of profiles
M = pd.concat([data1_x, data2_x, data3_x,data4_x, data5_x, data6_x],axis=1)
Z = pd.concat([data1_y, data2_y, data3_y,data4_y, data5_y,data6_y],axis=1)
names=list(M.columns)

empty_names=[]
for a in names:
    if len(M[a].dropna())<=3:
        empty_names.append(a)
        print (f"Appending {a}")
    else:
        continue 

## Fit B-spline to profile data



In [ ]:
x = []
y = []
k = []
# Define the parametric values at which the curvature will be evaluated, between 0 and 1. 
U = pd.Series(np.linspace(0,1,300))

# zip M_temp and Z_temp to create list of tuples of point
L=pd.DataFrame([list(zip(M[a].dropna(),Z[a].dropna())) for a in names]).transpose()
L.columns=names

# For each profile, fit the data with a B-spline, calculate its derivatives, calculate curvature.
for a in names:
    curve = BSpline.Curve()# define the BSpline curve
    curve.degree = 3# define the degree of the curve
    curve.ctrlpts=list(L[a].dropna())
    curve.knotvector = utilities.generate_knot_vector(curve.degree, len(curve.ctrlpts)) # auto-generate knot vector
    curve.evaluate() # evaluate curve
    # Calculate x,y coordinates and derivatives of curve at u values
    x_temp = []
    y_temp = []
    k_temp = []
    for i in U:
        ders = curve.derivatives(i, order = 2) # calculate 1st and 2nd derivatives of the curve at u, returns 3 tuples defined below.
        x_temp2 = ders[0][0] # x-position at u
        y_temp2 = ders[0][1] # y-position at u
        der1x_2 = ders[1][0] # 1st derivative with respect to x
        der1y_2 = ders[1][1] # 1st derivative with respect to y
        der2x = ders[2][0] # 2nd derivative with respect to x
        der2y = ders[2][1] # 2nd derivative with respect to y
        k_temp2 = ((der1x_2*der2y)-(der1y_2*der2x))/((((der1x_2)**2)+((der1y_2)**2))**(3/2)) # calculate curvature using formula for parametric curves.
        x_temp.append(x_temp2)
        y_temp.append(y_temp2)
        k_temp.append(k_temp2)
    x.append(x_temp)
    y.append(y_temp)
    k.append(k_temp)
x=pd.DataFrame(x).transpose()
x.columns=names
y=pd.DataFrame(y).transpose()
y.columns=names
k=pd.DataFrame(k).transpose()
k.columns=names

## Singular value decomposition

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
features=pd.DataFrame([pd.concat([x[a],y[a]]).reset_index(drop=True) for a in names]).transpose()
n_features, n_samples = features.shape
mean = pd.Series(np.mean(features, axis=1))
centered_data = pd.DataFrame([features[a]-mean for a in names]).transpose()
U, S, V = linalg.svd(centered_data,full_matrices=False)

In [ ]:
percent_variance = (S*100)/(S.sum())
fig,ax = plt.subplots()
ax.plot(percent_variance,marker='.')
ax.set_xlabel('Components')
ax.set_ylabel('Percent of variance explained by component')

In [ ]:
one = np.ones((len1,))
two = np.ones((len2,))*2
three = np.ones((len3,))*3
four = np.ones((len4,))*4
five = np.ones((len5,))*5
six = np.ones((len6,))*6

classes= np.concatenate((one,two,three,four,five,six), axis=0)

In [ ]:
proj=V.T*S
proj=pd.DataFrame(proj)

In [ ]:
x_ax=0
y_ax=1
z_ax=2

v11 = np.array(proj.iloc[0:len1,x_ax])
v12 = np.array(proj.iloc[0:len1,y_ax])
v13 = np.array(proj.iloc[0:len1,z_ax])
v21 = np.array(proj.iloc[len1:len1+len2,x_ax])
v22 = np.array(proj.iloc[len1:len1+len2,y_ax])
v23 = np.array(proj.iloc[len1:len1+len2,z_ax])
v31 = np.array(proj.iloc[len1+len2:len1+len2+len3,x_ax])
v32 = np.array(proj.iloc[len1+len2:len1+len2+len3,y_ax])
v33 = np.array(proj.iloc[len1+len2:len1+len2+len3,z_ax])
v41 = np.array(proj.loc[len1+len2+len3:len1+len2+len3+len4,x_ax])
v42 = np.array(proj.loc[len1+len2+len3:len1+len2+len3+len4,y_ax])
v43 = np.array(proj.loc[len1+len2+len3:len1+len2+len3+len4,z_ax])
v51 = np.array(proj.loc[len1+len2+len3+len4:len1+len2+len3+len4+len5,x_ax])
v52 = np.array(proj.loc[len1+len2+len3+len4:len1+len2+len3+len4+len5,y_ax])
v53 = np.array(proj.loc[len1+len2+len3+len4:len1+len2+len3+len4+len5,z_ax])
v61 = np.array(proj.loc[len1+len2+len3+len4+len5:len1+len2+len3+len4+len5+len6,x_ax])
v62 = np.array(proj.loc[len1+len2+len3+len4+len5:len1+len2+len3+len4+len5+len6,y_ax])
v63 = np.array(proj.loc[len1+len2+len3+len4+len5:len1+len2+len3+len4+len5+len6,z_ax])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.scatter(v11,v12,v13, c='firebrick',marker='o')
ax.scatter(v21,v22,v23,c='darkorange', marker='o')
ax.scatter(v31,v32,v33,c='darkmagenta', marker='o')
ax.scatter(v41,v42,v43,c='orchid', marker='o')
ax.scatter(v51,v52,v53,c='darkcyan', marker='o')
ax.scatter(v61,v62,v63,c='mediumslateblue', marker='o')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
#ax.set_title('Projection of data into principal component space')
plt.show()

### Train classifier

#### Optimize number of modes for SVM classification

In [ ]:
gamma=np.logspace(-9, 3, 13)
C=np.logspace(-3, 9, 13)
PC=list(range(1,total_len+1))

In [ ]:
product_C_gamma=list(itertools.product(C,gamma))
product_PC_C_gamma=list(itertools.product(PC,product_C_gamma))

In [ ]:
accuracy_optimization_svm=[]
for a in product_PC_C_gamma:
    R=a[0]
    K=a[1][0]
    g=a[1][1]
    
    X = proj.iloc[:,:R]
    scaler = preprocessing.StandardScaler().fit(X)
    X=scaler.transform(X)
    Y = classes
            #accuracy
    rbf = svm.SVC(kernel='rbf', C=K, gamma=g,decision_function_shape='ovr')
    scores = sklearn.model_selection.cross_val_score(rbf, X, Y, cv = 5)
        
    accuracy_optimization_svm_temp=np.mean(scores) 
    accuracy_optimization_svm.append(accuracy_optimization_svm_temp) 

In [ ]:
max(accuracy_optimization_svm)

In [ ]:
best_parameters=product_PC_C_gamma[accuracy_optimization_svm.index(max(accuracy_optimization_svm))]

In [ ]:
best_parameters

In [ ]:
r_best=best_parameters[0]
C_best=best_parameters[1][0]
gamma_best=best_parameters[1][1]

### Cross-validation SVM


In [ ]:
cv=5
f1Score=pd.DataFrame()
precision=pd.DataFrame()
recall=pd.DataFrame()
support=pd.DataFrame()
accuracy=pd.DataFrame()
cKappa=pd.DataFrame()
MCC=pd.DataFrame()

X = proj.iloc[:,0:r_best]
Y = classes

for a in range(0,cv): 
            # Split the data filenames into a training set and a testing set (67/33 split)
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y,test_size = 0.30)
    
            #classification
    classifier = svm.SVC(kernel='rbf', C=C_best,gamma=gamma_best, decision_function_shape='ovr')  
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
            
    #Cohen's kappa and Matthews correlation coefficient
    cKappaTemp=sklearn.metrics.cohen_kappa_score(y_test,y_pred)
    MCCTemp=sklearn.metrics.matthews_corrcoef(y_test,y_pred)

    #CLassification stats
    classReport=sklearn.metrics.classification_report(y_test,y_pred,output_dict=True)
    classification_Report=pd.DataFrame.from_dict(classReport)
    f1ScoreTemp=classification_Report.iloc[0,:]
    precisionTemp=classification_Report.iloc[1,:]
    recallTemp=classification_Report.iloc[2,:]
    supportTemp=classification_Report.iloc[3,:]
    accuracyTemp=sklearn.metrics.accuracy_score(y_test,y_pred)
    
    #Store info for the run into dataframes
    f1Score[a]=pd.Series(f1ScoreTemp)
    precision[a]=pd.Series(precisionTemp)
    recall[a]=pd.Series(recallTemp)
    support[a]=pd.Series(supportTemp)
    accuracy[a]=pd.Series(accuracyTemp)
    cKappa[a]=pd.Series(cKappaTemp)
    MCC[a]=pd.Series(MCCTemp)

classificationStats=pd.DataFrame(index=['1.0','2.0','3.0','4.0','5.0','6.0','macro avg','weighted avg'],columns=['f1Score','precision','recall','support','accuracy','Cohens Kappa','MCC'])
classificationStats.iloc[:,0]=f1Score.loc[f1Score.index != 'accuracy',:].mean(axis=1)
classificationStats.iloc[:,1]=precision.loc[precision.index != 'accuracy',:].mean(axis=1)
classificationStats.iloc[:,2]=recall.loc[recall.index != 'accuracy',:].mean(axis=1)
classificationStats.iloc[:,3]=support.loc[support.index != 'accuracy',:].mean(axis=1)
classificationStats.iloc[6,4]=accuracy.mean(axis=1)[0]
classificationStats.iloc[6,5]=cKappa.mean(axis=1)[0]
classificationStats.iloc[6,6]=MCC.mean(axis=1)[0]
classificationStats

### Visualizing principal components

In [ ]:
features=xy
n_samples, n_features = features.shape
mean = pd.Series(np.mean(features, axis=1))
centered_data = pd.DataFrame()
for a in names:
    centered_data_temp = features[a]-mean
    centered_data[a]=centered_data_temp
centered_data_t = centered_data.T

U, S, V = linalg.svd(features,full_matrices=False)

#### Mean profile form

In [ ]:
mean_2D=pd.DataFrame(mean.values.reshape((300,2)),columns=['x','y'])
plt.plot(mean_2D['x'],mean_2D['y'])
plt.title('Mean normalized scarp profile')

In [ ]:
PCX=pd.DataFrame(U[0::2,:r_best])
PCY=pd.DataFrame(U[1::2,:r_best])

In [ ]:
#PC1
PC1x = mean_2D['x']+PCX.iloc[:,0]
PC1y = mean_2D['y']+PCY.iloc[:,0]
fig,ax=plt.subplots()
ax.plot(PC1x,PC1y,'k',linewidth=2)
#ax.plot(x,y,'gray',linewidth=0.1)
ax.plot(mean_2D['x'],mean_2D['y'],'b:',linewidth=2)
ax.scatter(PC1x[260],PC1y[260])#,'k',linewidth=2)
ax.scatter(mean_2D['x'][260],mean_2D['y'][260])#,'k',linewidth=2)

#ax.plot(PCX.iloc[:,0],PCY.iloc[:,0],'r:')
ax.set_aspect('equal')
ax.set_title('PC1')

In [ ]:
#PC2
PC2x = mean_2D['x']+PCX.iloc[:,1]
PC2y = mean_2D['y']+PCY.iloc[:,1]
fig,ax=plt.subplots()
ax.plot(PC2x,PC2y,'k')
ax.plot(mean_2D['x'],mean_2D['y'],'b:')
ax.scatter(PC2x[100],PC2y[100])#,'k',linewidth=2)
ax.scatter(mean_2D['x'][100],mean_2D['y'][100])#,'k',linewidth=2)
ax.set_aspect('equal')
ax.set_title('PC2')

In [ ]:
mean_line=pd.Series((np.ones(300))*0)

In [ ]:
#PC3
PC3x = mean_2D['x']+PCX.iloc[:,2]
PC3y = mean_2D['y']+PCY.iloc[:,2]
fig,ax=plt.subplots()
ax.plot(PC3x,PC3y,'k')
ax.plot(mean_2D['x'],mean_2D['y'],'b:')
ax.set_title('PC3')

In [ ]:
#PC4
PC4x = mean_2D['x']+PCX.iloc[:,3]
PC4y = mean_2D['y']+PCY.iloc[:,3]
fig,ax=plt.subplots()
ax.plot(PC4x,PC4y,'k')
ax.plot(mean_2D['x'],mean_2D['y'],'b:')
ax.set_title('PC4')

In [ ]:
#PC5
PC5x = mean_2D['x']+PCX.iloc[:,4]
PC5y = mean_2D['y']+PCY.iloc[:,4]
fig,ax=plt.subplots()
ax.plot(PC5x,PC5y,'k')
ax.plot(mean_2D['x'],mean_2D['y'],'b:')
ax.scatter(PC5x[100],PC5y[100])#,'k',linewidth=2)
ax.scatter(mean_2D['x'][100],mean_2D['y'][100])
ax.set_aspect('equal')
ax.set_title('PC5')

In [ ]:
#PC6
PC6x = mean_2D['x']+PCX.iloc[:,5]
PC6y = mean_2D['y']+PCY.iloc[:,5]
fig,ax=plt.subplots()
ax.plot(PC6x,PC6y,'k')
ax.plot(mean_2D['x'],mean_2D['y'],'b:')
ax.set_title('PC6')

In [ ]:
#PC7
PC7x = mean_2D['x']+PCX.iloc[:,6]
PC7y = mean_2D['y']+PCY.iloc[:,6]
fig,ax=plt.subplots()
ax.plot(PC7x,PC7y,'k')
ax.plot(mean_2D['x'],mean_2D['y'],'b:')
ax.set_title('PC7')

In [ ]:
#PC8
PC8x = mean_2D['x']+PCX.iloc[:,7]
PC8y = mean_2D['y']+PCY.iloc[:,7]
fig,ax=plt.subplots()
ax.plot(PC8x,PC8y,'k')
ax.plot(mean_2D['x'],mean_2D['y'],'b:')
ax.set_title('PC8')